# Generate automated session reports for all participants

In [5]:
# import packages
import pandas as pd
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
import q1k_init_tools as qit
import glob
import warnings
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read
import os
import papermill as pm
warnings.filterwarnings('ignore')

### Select task parameters and file paths

In [6]:
# Select kind of data experimental or control group

dataset_group = "experimental"

#if dataset_group == "control":
#    # Control group data
#    project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
#    task_id_in = "ap"
#    task_id_in_et = "ap"
#    task_id_out = "ap"
#    #subject_id = '002'
#    session_id = '01'
#    run_id = '1'

#elif dataset_group == "experimental":
# Experimental group data

project_path = "/project/def-emayada/q1k/experimental/HSJ/"
init_path = "code/q1k_eeget_init/"
task_id_in = "RS"
task_id_in_et = "RS"
task_id_out = "RS"
#subject_id = 'Q1K_HSJ_100123_F1'
run_id = '1'
session_id = '01'
site_code = 'HSJ' #'MHC' or 'HSJ'
sourcedata_path = "sourcedata/" 
html_reports_path = "session_reports/" + task_id_in  + '/'
et_sync = True


In [7]:
# Sanity check to see which task you would like to make reports for 
print(task_id_in)
et_sync

RS


True

In [8]:
# Generate list of session reports already processed so they are not processed again in next step
processed_sessions = []

for subject in glob.glob(project_path + init_path + "session_reports/" + task_id_in + "/Q1k*.html"): 
    processed_sessions.append(subject.split('\\')[1][:-5])

print('Existing session reports:')
processed_sessions

Existing session reports:


[]

## Generate html session reports for all participants

In [9]:
glob.glob(project_path + sourcedata_path + "eeg/Q1K*/Q1K*" + task_id_in + '_*.mff')

['/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_100134_F1/Q1K_HSJ_100134_F1_RS_20240417_111808.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_100147_P/Q1K_HSJ_100147_P_RS_20240501_105202.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1045_P/Q1K_HSJ_1525-1045_P_RS_20240805_120946.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1001_F1/Q1K_HSJ_1525-1001_F1_RS_20240524_100502.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1026_P/Q1K_HSJ_1525-1026_P_RS_20240613_023013.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525_1052_P/Q1K_HSJ_1525_1052_P_RS_20240712_022009.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_100105_P/Q1K_HSJ_100105_P_RS_20240422_101342.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1026_M1/Q1K_HSJ_1525-1026_M1_RS_20240613_111502.mff',
 '/project/def-emaya

In [10]:
# Make sure the output directory exists
if not os.path.exists(html_reports_path):
    os.makedirs(html_reports_path)

# Create a list of sessions with errors
error_subjects = []

#Loop through existing sessions and execute the q1k_generate_individual_reports.ipynb if a session report does not already exist
for file in glob.glob(project_path + sourcedata_path + "eeg/Q1K*/Q1K*" + task_id_in + '_*.mff'):
    print('Current data file: ' + file)
    # Select anything after the Q1K and before the AEP
    #subject_id = file.split('_')[2]
    site_part = file.split(site_code + "_")
    subject_number = site_part[1].split("_")[0]
    print('Participant number: ' + subject_number)
    subject_relation = site_part[1].split("_")[1].split("/")[0]
    print('Participant relation: ' + subject_relation)

    subject_id_in = subject_number + "_" + subject_relation
    print('Participant ID input: ' + subject_id_in)
    subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
    print('Participant ID output: ' + subject_id_out)
    
    # Skip sessions that have a session report in the output directory
    print(subject_id_in)
    if subject_id_in in processed_sessions:
        print(subject_id_in + ' has already been processed')
        continue    

    # Define paths
    input_notebook = project_path + init_path +'q1k_generate_individual_reports.ipynb'
    print('Input notebook: ' + input_notebook)

    # Make sure the directory exists
    if not os.path.exists(f'{project_path}{init_path}session_reports/{task_id_in}/executed_notebooks/'):
        os.makedirs(f'{project_path}{init_path}session_reports/{task_id_in}/executed_notebooks/')
    
    output_notebook = f'{project_path}{init_path}{html_reports_path}executed_notebooks/{subject_id_in}_{task_id_in}_executed.ipynb'
    print('Output notebook file: ' + output_notebook)
    output_html = f'{project_path}{init_path}{html_reports_path}{subject_id_in}_{task_id_in}.html'
    print('Output HTML file: ' + output_html)

    try:
        # Execute the notebook
        pm.execute_notebook(input_notebook, output_notebook, kernel_name = 'q1k_env', parameters=dict(subject_id_in=subject_id_in, subject_id_out=subject_id_out,task_id_in=task_id_in, task_id_in_et=task_id_in_et,
         task_id_out=task_id_out, run_id=run_id, session_id=session_id, project_path=project_path, dataset_group=dataset_group, site_code=site_code))

        # Convert executed notebook to HTML
        html_exporter = HTMLExporter()
        html_exporter.exclude_input = True

        (body, resources) = html_exporter.from_filename(output_notebook)

        # Save HTML output
        with open(output_html, 'w', encoding='utf-8') as f:
            f.write(body)

        print(f"HTML report saved for {subject_id_in}.")
    

    except Exception as e:
        # Handle the error 
        error_subjects.append(subject_id_in)
        print(f"Error while processing {subject_id_in}: {e}")

# Print out the list of subjects with errors
print( "These subjects have errors: " + str(error_subjects) + " and need to be reprocessed")

Current data file: /project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_100134_F1/Q1K_HSJ_100134_F1_RS_20240417_111808.mff
Participant number: 100134
Participant relation: F1
Participant ID input: 100134_F1
Participant ID output: 100134F1
100134_F1
Input notebook: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/q1k_generate_individual_reports.ipynb
Output notebook file: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/session_reports/RS/executed_notebooks/100134_F1_RS_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/session_reports/RS/100134_F1_RS.html


Executing:   0%|          | 0/37 [00:00<?, ?cell/s]

HTML report saved for 100134_F1.
Current data file: /project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_100147_P/Q1K_HSJ_100147_P_RS_20240501_105202.mff
Participant number: 100147
Participant relation: P
Participant ID input: 100147_P
Participant ID output: 100147P
100147_P
Input notebook: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/q1k_generate_individual_reports.ipynb
Output notebook file: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/session_reports/RS/executed_notebooks/100147_P_RS_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/session_reports/RS/100147_P_RS.html


Executing:   0%|          | 0/37 [00:00<?, ?cell/s]

Error while processing 100147_P: 
---------------------------------------------------------------------------
Exception encountered at "In [29]":
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[29], line 3
      1 if et_sync:
      2     # peak... at the ET event scatter plot.. event time stamp by DIN index
----> 3     fig=px.scatter(x=et_events['index'],y=et_events['DIN_diff'])
      4 else:
      5     print('et_sync = False: skipping ET stimulus DIN offset display')

File /project/def-emayada/q1k/experimental/q1k_env/lib/python3.11/site-packages/plotly/express/_chart_types.py:66, in scatter(data_frame, x, y, color, symbol, size, hover_name, hover_data, custom_data, text, facet_row, facet_col, facet_col_wrap, facet_row_spacing, facet_col_spacing, error_x, error_x_minus, error_y, error_y_minus, animation_frame, animation_group, category_orders, labels, orientation, color_dis

...

Error while processing 1525-1057_F1: 
---------------------------------------------------------------------------
Exception encountered at "In [23]":
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[23], line 3
      1 #if et_sync:
      2     # peak... at the ET channel types
----> 3 et_types = et_raw.get_channel_types()
      4 np.unique(et_types)
      5 #else:
      6 #    print("et_sync = False: not printing ET channle types")

NameError: name 'et_raw' is not defined

Current data file: /project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_100162_P/Q1K_HSJ_100162_P_RS_20240530_114409.mff
Participant number: 100162
Participant relation: P
Participant ID input: 100162_P
Participant ID output: 100162P
100162_P
Input notebook: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/q1k_generate_individual_reports.ipynb
Output notebook file: /project/def-emaya

Executing:   0%|          | 0/38 [00:00<?, ?cell/s]

Error while processing 100162_P: 
---------------------------------------------------------------------------
Exception encountered at "In [30]":
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[30], line 3
      1 if et_sync:
      2     # peak... at the ET event scatter plot.. event time stamp by DIN index
----> 3     fig=px.scatter(x=et_events['index'],y=et_events['DIN_diff'])
      4 else:
      5     print('et_sync = False: skipping ET stimulus DIN offset display')

File /project/def-emayada/q1k/experimental/q1k_env/lib/python3.11/site-packages/plotly/express/_chart_types.py:66, in scatter(data_frame, x, y, color, symbol, size, hover_name, hover_data, custom_data, text, facet_row, facet_col, facet_col_wrap, facet_row_spacing, facet_col_spacing, error_x, error_x_minus, error_y, error_y_minus, animation_frame, animation_group, category_orders, labels, orientation, color_dis

Executing:   0%|          | 0/38 [00:00<?, ?cell/s]

Error while processing 100152_M1: 
---------------------------------------------------------------------------
Exception encountered at "In [28]":
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File /project/def-emayada/q1k/experimental/q1k_env/lib/python3.11/site-packages/pandas/core/indexes/base.py:3805, in Index.get_loc(self, key)
   3804 try:
-> 3805     return self._engine.get_loc(casted_key)
   3806 except KeyError as err:

File index.pyx:167, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:196, in pandas._libs.index.IndexEngine.get_loc()

File pandas/_libs/hashtable_class_helper.pxi:7081, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas/_libs/hashtable_class_helper.pxi:7089, in pandas._libs.hashtable.PyObjectHashTable.get_item()

KeyError: 'pupil_left'

The above exception was the direct cause of the following exception:

KeyError               

### Visualize all reports at once

In [ ]:
#for file in glob.glob("./../../sourcefiles/EEG/Q1K*/Q1K" + "*AEP" + '*.mff'):
#    # Select anything after the Q1K and before the AEP
#    subject_id = file.split('\\')[1]
#    # Skip sessions that have already been processed
#    print(subject_id)
#    if subject_id in processed_sessions:
#        print (subject_id + ' has already been processed')
#        continue    

#    # Handle participants with error 
#    try:
#        %run -i ./q1k_generate_individual_reports.ipynb --subject_id={subject_id}
#    except Exception as e:
#    # Handle the error (e.g., log it, skip participant, etc.)
#        print(f"Error while reading raw data for {subject_id}: {e}")
